# Ungraded lab 3: Another explanation about PCA

*Copyrighted material*

**Objectives:** Apply PCA to solve many problems

**Steps:**
* PCA to find a rotation matrix
* PCA to separate components of a mixture

<img src = 'pca.jpeg' width="width" height="height"/>

creds: Raunak Joshi

## Introduction

"Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables (entities each of which takes on various numerical values) into a set of values of linearly uncorrelated variables called principal components. This transformation is defined in such a way that the first principal component has the largest possible variance (that is, accounts for as much of the variability in the data as possible), and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to the preceding components. The resulting vectors (each being a linear combination of the variables and containing n observations) are an uncorrelated orthogonal basis set. PCA is sensitive to the relative scaling of the original variables.

PCA was invented in 1901 by Karl Pearson,[1] as an analogue of the principal axis theorem in mechanics; it was later independently developed and named by Harold Hotelling in the 1930s.[2] Depending on the field of application, it is also named the discrete Karhunen–Loève transform (KLT) in signal processing, the Hotelling transform in multivariate quality control, proper orthogonal decomposition (POD) in mechanical engineering, singular value decomposition (SVD) of X (Golub and Van Loan, 1983), eigenvalue decomposition (EVD) of XTX in linear algebra, factor analysis (for a discussion of the differences between PCA and factor analysis see Ch. 7 of Jolliffe's Principal Component Analysis),[3] Eckart–Young theorem (Harman, 1960), or empirical orthogonal functions (EOF) in meteorological science, empirical eigenfunction decomposition (Sirovich, 1987), empirical component analysis (Lorenz, 1956), quasiharmonic modes (Brooks et al., 1988), spectral decomposition in noise and vibration, and empirical modal analysis in structural dynamics."

<img src=GaussianScatterPCA.svg>

Source: https://en.wikipedia.org/wiki/Principal_component_analysis

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import math

To start lets consider a pair of random variables x, y. Lets start considering the base case when y = n * x. It is clear that x and y will be perfectly correlated to each other since y is just a scaling of x.

In [ ]:
n = 1
x = np.random.uniform(1,2,1000)
y = x.copy() * n

# PCA works better if the data is centered
x = x - np.mean(x)
y = y - np.mean(y)

data = pd.DataFrame({'x': x, 'y': y})
plt.scatter(data.x, data.y)

pca = PCA(n_components=2)
pcaTr = pca.fit(data)
dataPCA = pd.DataFrame(data = pcaTr.transform(data), columns = ['PC1', 'PC2'])

print(pca.components_.T)

plt.scatter(dataPCA.PC1, dataPCA.PC2)
plt.show()

Now, what is the direction in which the set of points are pointing the most?

## Undestanding the transformation model pcaTr

In [ ]:
# First Eigen Vector
print('Eigen vectors or principal component: First row must be in the direction of [1, n]')
print(pca.components_)

# First Eigen Value
print('Eigen values or explained variance')
print(pca.explained_variance_)


## Now lets introduce random variations to our correlated variables

Now, we will use a controlled dataset composed of 2 random variables with different variances and with a certain covariance amogh them. The only way I know to get such a dataset
is, first, create 2 independent random normal variables with the desired variances, and then combine them using a rotation matrix. In this way the new resulting variables will be a linear combination of the original random variables and thus be dependent and correlated.

In [ ]:
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

std1 = 1
std2 = 0.333

x = np.random.normal(0,std1,1000)
y = np.random.normal(0,std2,1000)
#y = y + np.random.normal(0,1,1000)*noiseLevel * np.sin(0.78)

# PCA works better if the data is centered
x = x - np.mean(x)
y = y - np.mean(y)

#Define a pair of dependent variables with a desired amount of covariance
n = 1 # Magnitude of covariance
angle = -np.arctan(1/n)
print('angle: ',  angle * 180 /math.pi)
rotationMatrix = [[np.cos(angle), -np.sin(angle)],
                 [np.sin(angle), np.cos(angle)]]


print('rotationMatrix ', rotationMatrix)
data = pd.DataFrame({'x': x, 'y': y})

print(data.iloc[0,:])
#plt.scatter(data.x, data.y)
# Rotation operation.
data = data.dot(rotationMatrix)
data.columns = ['x', 'y']
print(data.iloc[0,:])



In [ ]:
plt.scatter(data.x, data.y)

# Apply PCA. In theory, the Eigen Vector matrix must be the inverse of the original rotationMatrix. 
pca = PCA(n_components=2)
pcaTr = pca.fit(data)
dataPCA = pd.DataFrame(data = pcaTr.transform(data), columns = ['PC1', 'PC2'])
# First Eigen Vector
print('Eigen vectors or principal component: First row must be in the direction of [1, n]')
print(pca.components_)

# First Eigen Value
print('Eigen values or explained variance')
print(pca.explained_variance_)

plt.scatter(dataPCA.PC1, dataPCA.PC2)
plt.plot([0, rotationMatrix[0][0] * std1 * 3], [0, rotationMatrix[0][1] * std1 * 3], 'k-', color='red')
plt.plot([0, rotationMatrix[1][0] * std2 * 3], [0, rotationMatrix[1][1] * std2 * 3], 'k-', color='green')

plt.show()

## PCA as an strategy to reduce the dimensionality

The principal components contained in the rotation matrix, are decreasingly sorted depending on its explained variance. It usually means that the first components retains most of the power of the data to explain the pattenrs that generalize the data. Nevertheless, for some applications, we are really interested in the patters that explain much less variance. For example in novelty detection. In the figure we can see the original data, and its corresponding proyection over the first and second principal component.

In [ ]:
dataPCA1 = pd.DataFrame(data = pcaTr.transform(data), columns = ['PC1', 'PC2'])

plt.scatter(dataPCA.PC1, dataPCA.PC2)
plt.scatter(dataPCA.PC1, np.zeros(len(dataPCA.PC1)))
plt.scatter(np.zeros(len(dataPCA.PC2)), dataPCA.PC2)

plt.show()

## PCA as an strategy to plot complex data, as 2D or 3D points

One of my favoury uses of PCA is to create 2D or 3D charts out of very complex or highly dimensional data. We can find a very nice example in our cats/dogs classification where use use PCA to plot in a 2D chart the set of images used for classification. Remember that images are composed of pixels and each image is contains hundreds, thousans or millions of them. So, be able to map those complex objects in a 2D space is just awesome. Follow the link to view the example: [https://github.com/andcastillo/classification/blob/master/images/Classification.ipynb]

<img src = 'catdog.png'>


Still in progress. Sorry 😐 